In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [3]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

## Preprocess 

### Clothing Review Dataset

In [4]:
# Load dataset for clothing reviews
reviews = pd.read_csv("/content/drive/MyDrive/data /Womens Clothing E-Commerce Reviews.csv")
reviews = reviews.dropna()
print(reviews.shape)
reviews.head()

(19662, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [5]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

In [6]:
reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews.head()

,review,rating
2,Some major design flaws I had such high hopes ...,3
3,"My favorite buy! I love, love, love this jumps...",5
4,Flattering shirt This shirt is very flattering...,5
5,Not for the very petite I love tracy reese dre...,2
6,Cagrcoal shimmer fun I aded this in my basket ...,5


#### Tokenize each sentence

In [7]:
#take advantage of nltk to tokenize all sentences
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
en_stop_words = set(stopwords.words('english'))
tokenizer = nltk.RegexpTokenizer(r'\w+|\$[\d\.]+')

def tokenize_sent(sent):
    
    tokenized = tokenizer.tokenize(sent)
    filtered = [w.lower() for w in tokenized if w.lower() not in en_stop_words]
    return filtered

reviews['tokenized'] = reviews['review'].apply(lambda x: tokenize_sent(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
# Sanity cehck for tokenizers
print(reviews.head(3))

                                              review  ...                                          tokenized
2  Some major design flaws I had such high hopes ...  ...  [major, design, flaws, high, hopes, dress, rea...
3  My favorite buy! I love, love, love this jumps...  ...  [favorite, buy, love, love, love, jumpsuit, fu...
4  Flattering shirt This shirt is very flattering...  ...  [flattering, shirt, shirt, flattering, due, ad...

[3 rows x 3 columns]


#### Encode and truncate sentence

In [9]:
# get unique words in the corpus
all_words = []
for x in reviews['tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in reviews.tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in reviews['tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in reviews.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

reviews['encoded'] = reviews.tokenized.apply(lambda x: encode(x))

print(reviews.head())

# get sequence average length
total_len = 0
for x in reviews.encoded:
    total_len += len(x)

ave_len = math.floor(total_len/reviews.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

for i, row in reviews.iterrows():
    size = min(len(row.encoded), ave_len)
    new_encoded = row.encoded[:size]
    if size < ave_len:
        for j in range(0, ave_len - len(row.encoded)):
            new_encoded.append(1)
    reviews.at[i, 'encoded'] = new_encoded

reviews['review_length'] = reviews.encoded.apply(lambda x: len(x))

                                              review  ...                                            encoded
2  Some major design flaws I had such high hopes ...  ...  [1136, 5844, 8918, 8301, 11010, 2575, 7912, 10...
3  My favorite buy! I love, love, love this jumps...  ...  [958, 4960, 6105, 6105, 6105, 347, 714, 10086,...
4  Flattering shirt This shirt is very flattering...  ...  [13250, 9258, 9258, 13250, 2779, 9067, 4520, 5...
5  Not for the very petite I love tracy reese dre...  ...  [4970, 6105, 3780, 2507, 2352, 10476, 4970, 38...
6  Cagrcoal shimmer fun I aded this in my basket ...  ...  [2618, 4320, 714, 7950, 3389, 9512, 446, 1843,...

[5 rows x 4 columns]


In [10]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in reviews.encoded]
print(max(lengths))
print(min(lengths))
print(set(reviews.rating))
review_length = reviews.review_length
reviews.head()
print(len(word_set))

639434
32
32
32
{1, 2, 3, 4, 5}
14088


In [11]:
# Train test split from skearln
data_size = len(reviews['encoded'])
assert data_size == len(reviews['rating']) 
X, y = list(zip(list(reviews['encoded']),(list(reviews['review_length'])))), list(reviews['rating'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [12]:
# Sanity check for length match
print(X_train[0])

([6147, 7719, 1699, 10109, 14086, 7719, 2978, 4521, 4960, 9273, 6147, 10109, 8826, 3293, 700, 6105, 3826, 5771, 7789, 6117, 13628, 9273, 14088, 3618, 10109, 8826, 3205, 13405, 13773, 14088, 10886, 8826], 32)


## Dataset and Dataloader

### Clothing Review Dataset

In [13]:
class Dataset_Glove(Dataset):
    def __init__(self, X, y): 
        self.X = X
        y_new = [i-1 for i in y]
        self.y = y_new

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [14]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [15]:
print(trainSet_glv[0])

(([6147, 7719, 1699, 10109, 14086, 7719, 2978, 4521, 4960, 9273, 6147, 10109, 8826, 3293, 700, 6105, 3826, 5771, 7789, 6117, 13628, 9273, 14088, 3618, 10109, 8826, 3205, 13405, 13773, 14088, 10886, 8826], 32), 4)


In [16]:
def myCollate(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    # sort sequence according to it's length in a batch
    data = data
    # data.sort(key=lambda x: x[1], reverse = True)

    # data (review: list, length: int)
    review = torch.tensor([x[0] for x in data], dtype = torch.long)
    label = torch.tensor(label, dtype = torch.long)
    seq_len = [x[1] for x in data]
    return review, label, seq_len

In [17]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [18]:
trainLoader_glv = DataLoader(dataset = trainSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
testLoader_glv = DataLoader(dataset = testSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
it = iter(trainLoader_glv)
x, y, seq_len = it.next()

In [19]:
x.data

tensor([[ 9520,  4314, 13835,  5892,   994,  7435, 13249,  8219,  6105,  1766,
          7451,  4976, 12670,  9451, 10255,  8177,  8074,  7831,  6823, 10313,
          6823,   994, 10122, 10303,  5781,  3037, 10320, 10425,  3444,   994,
            89,  9935],
        [ 2575, 11814, 12768,  8177, 11683,  8405,  4970,  7809,  8426,  3826,
          8117,  3781,  6248,  5072,  4390, 12643, 11829,  4601, 10313, 12268,
          8405, 11331, 12268,    89,  8826, 11709, 12268, 10255, 12801, 11707,
         11683,  8405],
        [13773,  7727, 13824,  2575, 14080,  4160,  6026,  9295, 13250,  7384,
          2226, 13782,  9447,  2943,  1416,  2392, 13666,  2302,  2232,  3974,
          3033, 14084,  3647,  3826,  4572,  5838,  4860,  8119,  2672,  9400,
             1,     1],
        [ 9295, 14080, 10052, 11831, 10382,  8119, 10281,  4765, 12718,  6105,
          4227,  8177, 13250, 13824, 14080, 10052,  4442,  8962, 11703,  2129,
         10447,  2352, 13996,     1,     1,     1,     1,  

In [20]:
y

tensor([3, 4, 4, 4, 2, 2, 4, 4, 4, 4, 1, 4, 4, 3, 0, 4])

In [21]:
seq_len

[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]

## Embedding

In [22]:
def load_glove_vectors(glove_file="/content/drive/MyDrive/data /glove.6B.300d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors


In [23]:
# create embedding matrix
def emb_matrix(model, all_words, word2index):
    matrix_size = len(word_set) + 2
    emb_matrix = np.zeros((matrix_size, 300))
    emb_matrix[0] = np.random.uniform(-0.25, 0.25, 300) # vector for UNK
    emb_matrix[1] = np.zeros(300) # vector for padding, has no weight

    # vector for every other word in the dictionary
    for i in range(2, len(all_words)+2):
        word = all_words[i-2]
        index = word2index[word]
        try:
            vector = model[word]
        except Exception as e:
            # word does not exist in the pretrained embedding
            vector = emb_matrix[0]
        emb_matrix[index] = vector
    return emb_matrix

In [24]:
glv = load_glove_vectors()
data = list(glv.values())
an_array=np.array(data)

In [25]:
embedding = emb_matrix(glv, word_set, word2index)

In [26]:
print(embedding.shape)

(14090, 300)


In [27]:
print(embedding)

[[ 0.15430956  0.08365842 -0.10828335 ...  0.05460924  0.19107885
   0.14338955]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.35896    -0.63622    -0.24773    ...  0.43317     0.3965
   0.13253   ]
 ...
 [ 0.10107     0.57044    -0.12155    ...  0.41924     0.1839
   0.095753  ]
 [-0.25641    -0.27075    -0.01199    ... -0.34087    -0.3813
   0.13515   ]
 [ 0.27571    -0.47332    -0.30355    ...  0.44401     0.18273
   0.3576    ]]


In [28]:
def validation_metrics(model, valid_dl):
    print("current in vlaidaiton")
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    y_total = []
    y_pred_total = []
    for x, y, seq_len in valid_dl:
        x = x.long().to(device)
        y = y.long().to(device)
        y_hat = model(x, seq_len)
        loss = criterion(y_hat, y)
        
        # print(f"shape of y: {y.shape}")
        # print(f"shape of y_pred: {y_hat.shape}")
        # print(y_hat)
        pred = torch.argmax(y_hat, 1)
        y_total.extend(y.tolist())
        y_pred_total.extend(pred.tolist())
        
        # print(pred)
        correct += sum((pred == y))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        # print(len(y_total))
        # print(len(y_pred_total))

      # calculate precision, recall, and f1
    f1 = (f1_score(y_total, y_pred_total, average='weighted'))
    precision = (precision_score(y_total, y_pred_total, average='weighted'))
    recall =(recall_score(y_total, y_pred_total, average='weighted'))
    return sum_loss/total, correct/total, precision, recall, f1

In [29]:
def train_model(model, fname, epochs = 50, lr = 0.0001):
    parameters = filter(lambda p:p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr = lr)
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    PATH = '/content/drive/MyDrive/McGill/Comp 550/' + fname
    epoch = 0
    best_val_acc = 0.0
    for i in range(epochs):
        epoch += 1
        print(f"At epoch {i}")
        batch_count = 0
        model.train()
        sum_loss = 0.0
        total = 0.0
        train_correct = 0.0
        for x, y, seq_len in tqdm(trainLoader_glv):
            x = x.long().to(device)
            y = y.long().to(device)
            batch_count += 1
            optimizer.zero_grad()
            y_pred = model(x, seq_len)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, precision, recall, f1 = validation_metrics(model, testLoader_glv)



        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f, precision %.3f, recall %.3f, F1 %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc, precision, recall, f1))
        if val_acc > best_val_acc and i>=2:
            best_val_acc = val_acc
            NEW_PATH = PATH+f'_{i}_{val_acc}'
            #torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")

In [30]:
weight = torch.FloatTensor(an_array)
glv_embedding = nn.Embedding.from_pretrained(weight)

class LSTM_glv(torch.nn.Module):
    def __init__(self, emb_matrix, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(emb_matrix.shape[0], embedding_dim, padding_idx = 0)
        self.embeddings.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, seq_len):
        x = self.embeddings(x)
        # x = nn.utils.rnn.pack_padded_sequence(x, self.embedding_dim, batch_first = True)
        x = self.dropout(x)
        # x_padded = pack_padded_sequence(x, seq_len, batch_first = True)
        output, (h_n, c_n) = self.lstm(x)
        # print(h_n[-1].shape)
        # output (N, ave_len, num_class)
        # reshape to out (N, num_class)
        
        return self.linear(h_n[-1])

In [31]:
model = LSTM_glv(embedding, 300, 100)
train_model(model, fname='glv_lstm', epochs=100)

At epoch 0


100%|██████████| 983/983 [00:03<00:00, 301.59it/s]


current in vlaidaiton


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss 1.195, train accuracy 0.532, val loss 1.078, val accuracy 0.555, precision 0.434, recall 0.555, F1 0.413
At epoch 1


...

current in vlaidaiton
train loss 0.169, train accuracy 0.938, val loss 1.787, val accuracy 0.629, precision 0.607, recall 0.629, F1 0.616
At epoch 98


100%|██████████| 983/983 [00:03<00:00, 312.32it/s]


current in vlaidaiton
train loss 0.163, train accuracy 0.939, val loss 1.824, val accuracy 0.629, precision 0.608, recall 0.629, F1 0.616
At epoch 99


100%|██████████| 983/983 [00:03<00:00, 310.94it/s]


current in vlaidaiton
train loss 0.165, train accuracy 0.941, val loss 1.810, val accuracy 0.628, precision 0.597, recall 0.628, F1 0.608
